In [1]:
#pip install chromadb==0.5.3 因为chroma 0.5.4 version有报错AttributeError: 'Collection' object has no attribute 'model_fields'
import os
from dotenv import load_dotenv
#加载 .env文件中的环境变量,保存在代码当前路径下
load_dotenv()
import openai
from langchain_openai import OpenAI
from langchain.schema import AIMessage,SystemMessage,HumanMessage
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import json

llm= ChatOpenAI(model_name="chatglm3-6b",temperature=0.1)

import chromadb
from chromadb.utils import embedding_functions
# 指定chromadb提供的embedding function使用openai接口能力
bge_embeddings = embedding_functions.OpenAIEmbeddingFunction(
    api_base=os.environ["OPENAI_API_BASE"],
    api_key=os.environ["OPENAI_API_KEY"],
    model_name="bge-large-zh-1.5"
)

In [2]:
r=llm.predict("hello")
print(r)

c:\Users\lenovo\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Hello! How can I help you today?


In [8]:
#用python内置的with open语句 读写文件是最常见的IO操作
with open("c:\\Users\\lenovo\\生活百科.txt") as file:
    test_text = file.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
#用三个回车作为分隔符，将预处理后的文本文件切成小段
text_splitter = CharacterTextSplitter(
    separator = '\n\n\n',
    chunk_size=450,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
chunks = text_splitter.split_text(test_text)

Created a chunk of size 537, which is longer than the specified 450
Created a chunk of size 582, which is longer than the specified 450
Created a chunk of size 478, which is longer than the specified 450
Created a chunk of size 470, which is longer than the specified 450
Created a chunk of size 586, which is longer than the specified 450
Created a chunk of size 1050, which is longer than the specified 450
Created a chunk of size 583, which is longer than the specified 450
Created a chunk of size 479, which is longer than the specified 450
Created a chunk of size 510, which is longer than the specified 450
Created a chunk of size 455, which is longer than the specified 450
Created a chunk of size 545, which is longer than the specified 450
Created a chunk of size 556, which is longer than the specified 450
Created a chunk of size 591, which is longer than the specified 450
Created a chunk of size 715, which is longer than the specified 450
Created a chunk of size 556, which is longer th

In [9]:
len(chunks)

106

In [10]:
#创建持久化数据库
dbclient = chromadb.PersistentClient(path="d:\\chromadb")

In [14]:
#测试，如有需要，删除数据库里的实验用cellection，清除之前的实验内容
dbclient.delete_collection("cosine_test")

In [11]:
#将文本块导入向量数据库的名称是“RAG_test”的cellection
cos_collection = dbclient.get_or_create_collection(name="cosine_test", 
                                               metadata={"hnsw:space": "cosine"}, # l2 is the default
                                               embedding_function=bge_embeddings) 

In [16]:
#按chunk排位作为ID
ids=[str(i) for i in range(len(chunks))]

In [20]:
#将原文的文字块存入向量数据库
cos_collection.add(ids=ids,documents=chunks)

Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
In

In [13]:
#基于query检索出5个最相似的document
query = "用微波炉制作菜肴的方法"

results=cos_collection.query(query_texts=[query],n_results=5)
retrieved_documents=results['documents'][0]

In [14]:
for retrieved_document in retrieved_documents:
    print(retrieved_document+"\n\n")

用微波炉做美容保健粥

    原料：半碗大米、一小碗熟玉米粒、一小碗去皮毛豆和20枸杞子。盐，糖。

    作法：把米淘洗干净，枸杞子用冷水洗净。将枸杞子用温水泡至回软，把淘好的米和去皮毛豆放入容器中，容器中加入水，容器加盖后放入微波炉，高波火力烹调8分钟。8分钟后将容器取出，将玉米加入容器中，。在容器中加入2勺糖和少半勺盐,用勺搅匀,容器加盖后放入微波炉，继续用高波火力烹调5分钟。5分钟后将容器取出，放入枸杞子拌匀，美容保健粥就做成啦。


微波炉烹制菊花粥

    原料：菊花、枸杞、糯米、冰糖

    步骤：

    1、糯米用水洗净，放大碗里，接一碗水，加盖放进微波炉，高火8分钟；

    2、菊花用擀面杖碾碎、去蒂；枸杞用温水浸泡10分钟；

    3、8分钟过去，取出半成品粥，加入适量枸杞，再入微波炉，中火5分钟；

    4、5分钟后加入碾碎的菊花，再中火2分钟

    5、取出放入冰糖


剁椒鱼头

    原料：胖头鱼鱼头一个，超市购买的剁辣椒、鲜红椒，小葱，大蒜和生姜，还有就是蚝油。首先把鱼头从上面切开，把蚝油倒在小碗里，然后涂抹在鱼头表面，把红辣椒、小葱洗干净，切成小丁，姜切成姜末，蒜也切成丁，现在舀出四勺剁辣椒，加上一点盐增加剁椒的味道，把它拌匀，把蒜末和姜末也放进剁椒里。然后把剁椒撒在鱼头上，盖上盖子腌制一个小时。再倒入两勺料酒，一勺食醋，少许鸡精，调成汁淋在腌好的鱼头上。

    进微波炉之前往鱼头上淋少许食用油，盖上盖子，放进微波炉里，调中火，定时十分钟。十分钟后拿出鱼头，把鱼头翻个个，把剁辣椒舀到鱼头的另一面，然后再盖上盖子，放进微波炉里，中火，微波四分钟。

    在鲜红椒里放少许盐，拌匀。

    时间到，我们拿出鱼头，把鲜红椒撒在鱼头上，再倒半碗水，放入葱花，稍微拌一下，盖上盖子，再微波三分钟，三分钟后就完成了把鱼头从微波炉里拿出来，香辣味顿时扑面而来您不妨也试试吧


微波炉做丝糕

    材料：白面、玉米面、小米面、泡打粉、发酵粉、鸡蛋、白砂糖、葡萄干、枸杞、黑芝麻。

    作法：面粉共250克，白面、玉米面、小米面各占13。把白面、玉米面、小米面放入一个盆里，搅拌均匀，然后，放入这样一小勺泡打粉。接下来，在小碗里倒入一小勺发酵粉，用温水搅拌均匀。然后，倒入面盆里，再倒入适量的温水，不停的搅拌，

In [15]:
#极简RAG，直接使用检索结果作为提示词上下文传给LLM生成答案
def rag(query,retrieved_documents):
    information= "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "你是一个乐于回答问题的助手，你的用户会问各种生活常识类问题。"
            "用户的提问和相关参考资料会一并提供给你，请仅用参考资料中的内容做答。"
        },
        {
            "role": "user",
            "content": f"用户提问：{query}。\n 参考资料：{information}"
        }
    ]

    response = llm.invoke(messages)
    content= response.content
    return content

In [16]:
output = rag(query=query,retrieved_documents=retrieved_documents)

print(output)

使用微波炉制作菜肴的方法如下：

1. 制作美容保健粥：半碗大米、一小碗熟玉米粒、一小碗去皮毛豆和20枸杞子。先将米和去皮毛豆放入容器中，加水炖煮8分钟，然后加入玉米粒和枸杞子，再用微波炉高火烹调5分钟即可。

2. 制作菊花粥：将菊花、枸杞、糯米、冰糖放入容器中，先用微波炉高火烹调8分钟，然后加入适量枸杞，再次用微波炉中火烹调5分钟。

3. 制作剁椒鱼头：将胖头鱼鱼头、剁辣椒、鲜红椒、小葱、大蒜和生姜放入容器中，加入蚝油，搅拌均匀后腌制一个小时。将腌制好的鱼头放入微波炉，用中火烹调10分钟，再加入剁辣椒，继续用中火烹调2分钟。最后加入鲜红椒、葱花等调料，再盖上盖子，用微波炉中火烹调3分钟即可。

4. 制作丝糕：将白面、玉米面、小米面、泡打粉、发酵粉、鸡蛋、白砂糖、葡萄干、枸杞、黑芝麻放入一个 bowl里，搅拌均匀后，放入微波炉专用玻璃碗，撒上葡萄干、枸杞和黑芝麻，用高火微波8分钟即可。

5. 制作微波炉鸡翅：将洗净的鸡翅放入盘中，撒上盐、酱油、味精、胡椒粉、鸡粉、淀粉，搅拌均匀后腌制5分钟。然后在腌好的鸡翅上面撒上胡椒粉，再搅拌一下。把鸡翅放到微波炉专用玻璃碗里，用高火微波5分钟即可。


In [17]:
#基于问题生成样例答案
def augement_query_generation(query):

    messages = [
        {
            "role": "system",
            "content": "你是一个乐于回答问题的助手，你的用户会问各种生活常识类问题。"
            "请基于用户问题生成3个不同类型答案样例，此答案样例将用于参考资料查找。"
            "输出格式是每个答案样例单独一行，不需要序号或说明。"
        },
        {
            "role": "user",
            "content": f"用户提问：{query}。"
        }
    ]

    response=llm.invoke(messages) 
    content= response.content
    return content

In [18]:
#基于query和样例答案，检索出5个最相似的document
query = "用微波炉制作菜肴的方法"
hyothetical_answer=augement_query_generation(query)
joint_query=f"{query} \n\n {hyothetical_answer}"
joint_query=joint_query.split("\n\n")

results=cos_collection.query(query_texts=joint_query,n_results=10)
retrieved_documents=results['documents'][0]

In [19]:
print(joint_query)
for retrieved_document in retrieved_documents: 
    print(retrieved_document+"\n\n")

['用微波炉制作菜肴的方法 ', ' 1. 先将食材洗净，切成适当大小的块或片。\n2. 将平底锅或微波炉专用烤盘放入微波炉中，加热至适当的温度。\n3. 将切好的食材放入锅中，用中低火或高速档加热至熟透即可。\n4. 如果需要，可以加入调料如盐、胡椒粉等，搅拌均匀即可享用。']
用微波炉做美容保健粥

    原料：半碗大米、一小碗熟玉米粒、一小碗去皮毛豆和20枸杞子。盐，糖。

    作法：把米淘洗干净，枸杞子用冷水洗净。将枸杞子用温水泡至回软，把淘好的米和去皮毛豆放入容器中，容器中加入水，容器加盖后放入微波炉，高波火力烹调8分钟。8分钟后将容器取出，将玉米加入容器中，。在容器中加入2勺糖和少半勺盐,用勺搅匀,容器加盖后放入微波炉，继续用高波火力烹调5分钟。5分钟后将容器取出，放入枸杞子拌匀，美容保健粥就做成啦。


微波炉烹制菊花粥

    原料：菊花、枸杞、糯米、冰糖

    步骤：

    1、糯米用水洗净，放大碗里，接一碗水，加盖放进微波炉，高火8分钟；

    2、菊花用擀面杖碾碎、去蒂；枸杞用温水浸泡10分钟；

    3、8分钟过去，取出半成品粥，加入适量枸杞，再入微波炉，中火5分钟；

    4、5分钟后加入碾碎的菊花，再中火2分钟

    5、取出放入冰糖


剁椒鱼头

    原料：胖头鱼鱼头一个，超市购买的剁辣椒、鲜红椒，小葱，大蒜和生姜，还有就是蚝油。首先把鱼头从上面切开，把蚝油倒在小碗里，然后涂抹在鱼头表面，把红辣椒、小葱洗干净，切成小丁，姜切成姜末，蒜也切成丁，现在舀出四勺剁辣椒，加上一点盐增加剁椒的味道，把它拌匀，把蒜末和姜末也放进剁椒里。然后把剁椒撒在鱼头上，盖上盖子腌制一个小时。再倒入两勺料酒，一勺食醋，少许鸡精，调成汁淋在腌好的鱼头上。

    进微波炉之前往鱼头上淋少许食用油，盖上盖子，放进微波炉里，调中火，定时十分钟。十分钟后拿出鱼头，把鱼头翻个个，把剁辣椒舀到鱼头的另一面，然后再盖上盖子，放进微波炉里，中火，微波四分钟。

    在鲜红椒里放少许盐，拌匀。

    时间到，我们拿出鱼头，把鲜红椒撒在鱼头上，再倒半碗水，放入葱花，稍微拌一下，盖上盖子，再微波三分钟，三分钟后就完成了把鱼头从微波炉里拿出来，香辣味顿时扑面而来您不妨也试试吧


微波炉做丝糕

    材料：白面、玉米面、小米面、泡打粉

In [20]:
#定义用httpx链接rerank接口
import httpx
from pydantic.main import BaseModel

RERANK_URL = "{url}/bge_base_rerank"
url=RERANK_URL.format(url=os.environ["OPENAI_API_BASE"])
headers = {'Content-Type': 'application/json'}

class QuerySuite(BaseModel):
    query: str
    passages: list[str]
    top_k: int = 5

def rerank(query_suite: QuerySuite):


    r = httpx.post(url, headers=headers, json=query_suite)

    return r.json()

In [24]:
import numpy as np

pairs = {'query':query,'passages':retrieved_documents}
query_results = rerank(pairs)

for key, value in query_results.items():
    print(f"score:{value} \n {key} \n\n")
# for query_result in query_results:
#     print(query_result+"\n\n")

score:2.875 
 微波炉鸡翅

    如果你是一个繁忙的上班族，那么下面这道菜你可就要看好了，因为可以让你学会五分钟用微波炉做出香喷喷的鸡翅来，即简捷又快速。

    首先准备好鸡翅和佐料，有盐、酱油、味精、胡椒粉、鸡粉、淀粉。

    把洗净的鸡翅放到盘中，撒上盐，到入酱油，再放入味精，搅拌均匀，让鸡翅的正反面都可以粘到佐料，然后腌制5分钟。

    5分钟后，在腌好的鸡翅上面撒上胡椒粉，然后再搅拌一下。下面把鸡粉和淀粉和huo在一起，调匀。调匀后，把鸡翅放到上面，让鸡翅的正反面都粘上鸡粉和淀粉的混合物。下面要做的就是把鸡翅放到微波炉里面进行加热，高火，5分钟。

    香喷喷的鸡翅做好了，是不是很省事呢，如果你抵挡不住诱惑那就快点自已动手品尝美味吧。 


score:2.67578125 
 730串烧鸡柳制作法

    原料：鸡腿肉半斤，菠萝2片，蘑菇一两，青椒1个，还有洋葱半个，蒜头2粒。料酒、蒜茸辣酱、生抽、糖、胡椒粉、花生油。

    作法：将蒜切碎，把蒜和料酒、蒜茸辣酱、生抽、糖、胡椒粉还有麻油倒在一个容器里面，把鸡腿肉泡到调好的调料当中，浸泡二十分钟。

    盘子加热之后，在煎盘里倒入一勺量左右的食用油，然后把串好的肉串放入盘子里，然后用烧烤模式下的高火加热五分钟。

    步骤回顾：

    第一步：把鸡腿肉切成两厘米左右的小肉块。

    第二步：把鸡腿肉泡到调好的调料当中，浸泡二十分钟。

    第三步：菠萝、青椒、洋葱和蘑菇切成小方块。

    第四步：以上下是鸡肉，中间是蔬菜的排列方式把原料串在竹签上。

    第五步：将微波炉盘放在微波炉中加热两分钟，之后盘中加一勺油。

    第六步：把串好的肉串放进微波炉高火烘烤五分钟。 


score:2.6015625 
 微波炉嫩蛋羹

    蒸蛋羹许多人掌握不好火候，不是蒸老了，就是没熟。用微波炉更不好掌握了。还是接着看朱奶奶怎么做吧。

    其实，蒸鸡蛋羹嫩不嫩的关键，就是往鸡蛋里倒入热水，水和鸡蛋的比率是2比1。接着朝一个方向搅拌均匀。然后，就可以放微波炉里了。放进入时容器上可以不盖盖儿。用高火，加热2分20秒左右。加热后，尽量远离微波炉，因为微波炉有微量的辐射。

    在加热期间，朱奶奶在调制鸡蛋羹的调料。在碗里倒入适量的醋、酱油、